In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pylab as plt
import sys, gc, warnings, random, math, time, datetime 
from tqdm import tqdm
START_DATE = datetime.datetime.strptime('2017-11-30', '%Y-%m-%d')

from sklearn import preprocessing
from sklearn import metrics
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
from scipy.stats.stats import pearsonr

import xgboost as xgb
import lightgbm as lgb

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

from utils import *

import os
print(os.listdir("../input/mini-model-data/"))

['test_df_h.pkl', 'test_df_c.pkl', 'train_df_w.pkl', 'test_df_w.pkl', 'train_df_s.pkl', 'test_df_s.pkl', 'test_df_r.pkl', 'train_df_h.pkl', 'test_identity.pkl', 'train_df_r.pkl', 'train_identity.pkl', 'train_df_c.pkl']


In [2]:
train_df = pd.read_pickle('../input/mini-model-data/train_df_r.pkl')
test_df = pd.read_pickle('../input/mini-model-data/test_df_r.pkl')

train_identity = pd.read_pickle('../input/mini-model-data/train_identity.pkl')
test_identity = pd.read_pickle('../input/mini-model-data/test_identity.pkl')

train = train_df.merge(train_identity, how='left', left_index=True, right_index=True)
test = test_df.merge(test_identity, how='left', left_index=True, right_index=True)

train_y = train_df['isFraud'].copy()

# print(train_df.shape)
# print(test_df.shape)
# print(train_identity.shape)
# print(test_identity.shape)
print(train.shape)
print(test.shape)

del train_df, test_df, train_identity, test_identity

(37699, 433)
(35647, 433)


In [3]:
train.head()

,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
TransactionID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2987099,0,88162,75.0,R,1214,174.0,150.0,visa,226.0,credit,126.0,87.0,NaN,NaN,gmail.com,gmail.com,2.0,1.0,0.0,2.0,0.0,1.0,0.0,1.0,0.0,2.0,1.0,0.0,2.0,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,12.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,0.0,125.0,0.0,0.0,125.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,125.0,0.0,0.0,0.0,0.0,125.0,125.0,125.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,0.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0

In [4]:
target_encode_book = [
    'addr1', 'C1', 'C2', 'C4', 'C6', 'C8', 'C10', 'C11', 'C13', 'C14', 
    'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'D9',
    'device_name', 'DeviceType',
    'id_12', 'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_19',
    'id_20', 'id_23', 'id_24', 'id_25', 'id_26', 'id_28', 'id_29',
    'id_30_device', 'id_30_version', 'lastest_browser', 
    'id_32', 'id_33', 'id_33_0', 'id_33_1', 'id_33_area', 'id_33_ratio','id_35', 'id_37', 'id_38',
    'P_emaildomain','R_emaildomain', 'cents'
]
# CV: C13, C14

In [5]:
def fe_dt(df):
    
    df['DT'] = df['TransactionDT'].apply(
        lambda x: (START_DATE + datetime.timedelta(seconds=x)))  # tmp    
    df['DT_month'] = (df['DT'].dt.year-2017)*12 + df['DT'].dt.month
    df['DT_week_year'] = (df['DT'].dt.year-2017)*52 + df['DT'].dt.weekofyear
    df['DT_day_year'] = (df['DT'].dt.year-2017)*365 + df['DT'].dt.dayofyear
    
    df['DT_hour'] = df['DT'].dt.hour
    df['DT_day_week'] = df['DT'].dt.dayofweek
    df['DT_day'] = df['DT'].dt.day
    
    for col in ['D1', 'D2', 'D3', 'D6', 'D7', 'D8','D10', 'D13','D14', 'D15']:
        # day map
        day_temp_dict = df.groupby('DT_day_year')[col].agg(['mean']).rename(columns={'mean':col + '_day_mean'})
        day_temp_dict = day_temp_dict[col + '_day_mean'].to_dict()
        df[col + '_day_mean'] = df['DT_day_year'].map(day_temp_dict)
        df[col + '_day_mean'] = df[col] - df[col + '_day_mean']
        
        # week map
        week_temp_dict = df.groupby('DT_week_year')[col].agg(['mean']).rename(columns={'mean':col + '_week_mean'})
        week_temp_dict = week_temp_dict[col + '_week_mean'].to_dict()
        df[col + '_week_mean'] = df['DT_week_year'].map(week_temp_dict)
        df[col + '_week_mean'] = df[col] - df[col + '_week_mean']

    df = df.drop(['DT'],axis=1)
    
    return df

In [6]:
def fe_uid(train_df, test_df):
    
    train_df['uid'] = train_df['card1'].astype(str)+'_'+train_df['card2'].astype(str)
    test_df['uid'] = test_df['card1'].astype(str)+'_'+test_df['card2'].astype(str)

    train_df['uid2'] = train_df['uid'].astype(str)+'_'+train_df['card3'].astype(str)+'_'+train_df['card5'].astype(str)
    test_df['uid2'] = test_df['uid'].astype(str)+'_'+test_df['card3'].astype(str)+'_'+test_df['card5'].astype(str)

    train_df['uid3'] = train_df['uid2'].astype(str)+'_'+train_df['addr1'].astype(str)+'_'+train_df['addr2'].astype(str)
    test_df['uid3'] = test_df['uid2'].astype(str)+'_'+test_df['addr1'].astype(str)+'_'+test_df['addr2'].astype(str)

    # Check if the Transaction Amount is common or not (we can use freq encoding here)
    # In our dialog with a model we are telling to trust or not to these values   
    train_df['TransactionAmt_check'] = np.where(train_df['TransactionAmt'].isin(test_df['TransactionAmt']), 1, 0)
    test_df['TransactionAmt_check']  = np.where(test_df['TransactionAmt'].isin(train_df['TransactionAmt']), 1, 0)

    i_cols = ['card1','card2','card3','card5','uid','uid2','uid3']

    for col in i_cols:
        for agg_type in ['mean','std']:
            new_col_name = col+'_TransactionAmt_'+agg_type
            temp_df = pd.concat([train_df[[col, 'TransactionAmt']], test_df[[col,'TransactionAmt']]])
            #temp_df['TransactionAmt'] = temp_df['TransactionAmt'].astype(int)
            temp_df = temp_df.groupby([col])['TransactionAmt'].agg([agg_type]).reset_index().rename(
                                                    columns={agg_type: new_col_name})

            temp_df.index = list(temp_df[col])
            temp_df = temp_df[new_col_name].to_dict()   

            train_df[new_col_name] = train_df[col].map(temp_df)
            test_df[new_col_name]  = test_df[col].map(temp_df)

    train_df['TransactionAmt'] = np.log1p(train_df['TransactionAmt'])
    test_df['TransactionAmt'] = np.log1p(test_df['TransactionAmt'])   
    
    train_df = train_df.drop(['uid','uid2','uid3'],axis=1)
    test_df = test_df.drop(['uid','uid2','uid3'],axis=1)
    
    train_df = train_df.replace(np.inf, 999)
    test_df = test_df.replace(np.inf, 999)
    
    return train_df, test_df

In [7]:
# feature engineering

def fe(train, test):

    drop_na_cols = []
    for col in train.columns:
        if (len(train[col].dropna().unique()) <= 1):
            drop_na_cols.append(col)
    print(len(drop_na_cols), ' features are removed...')
    # remove all nan columns
    train = train.drop(drop_na_cols, axis=1)
    test = test.drop(drop_na_cols, axis=1)
    
    # user id
    train, test = fe_uid(train, test)

    ignore_cols = ['P_emaildomain', 'R_emaildomain', 'DeviceInfo', 'id_30', 'id_31', 'id_33']
    for col in train.columns:
        if (train[col].dtype == 'object' or test[col].dtype == 'object') and col not in ignore_cols:
            lbl = preprocessing.LabelEncoder()
            lbl.fit(list(train[col].values) + list(test[col].values))
            train[col] = lbl.transform(list(train[col].values))
            test[col] = lbl.transform(list(test[col].values))
    
    for df in [train, test]:
        # device info
        df = setDevice(df)
        
        # id_30
        df['id_30'] = df['id_30'].fillna('unknown_device').str.lower()
        df['id_30_device'] = df['id_30'].apply(
            lambda x: ''.join([i for i in x if i.isalpha()]))
        df['id_30_version'] = df['id_30'].apply(
            lambda x: ''.join([i for i in x if i.isnumeric()]))
        
        # id_31
        df = setBrowser(df)
        
        # nan encoding
        for col in train.columns:
            if(train[col].isnull().sum() / len(train) > 0.2):
                df[col + '_isnull'] = df[col].isnull() + 0
        
        # email map
        for item in ['P_emaildomain', 'R_emaildomain']:
            df[item] = df[item].map(emails)
            
        # id_33
        df['id_33'] = df['id_33'].fillna('0x0')
        df['id_33_0'] = df['id_33'].apply(lambda x: x.split('x')[0]).astype(int)
        df['id_33_1'] = df['id_33'].apply(lambda x: x.split('x')[1]).astype(int)
        df['id_33'] = np.where(df['id_33']=='0x0', np.nan, df['id_33'])
        df['id_33_area'] = df['id_33_0'] * df['id_33_1']
        df['id_33_ratio'] = df['id_33_0'] / df['id_33_1']
        
        # cents
        df['cents'] = df['TransactionAmt'].apply(lambda x: x - int(x))
        

    train['cents'] = train['TransactionAmt'].apply(lambda x: x - int(x))
    test['cents'] = test['TransactionAmt'].apply(lambda x: x - int(x))
    
    for col in target_encode_book:
        train[col+'_target_code'], test[col+'_target_code'] = target_encode(train[col], test[col], train_y,min_samples_leaf=100, noise_level=0.01)
        
        temp_df = pd.concat([train[[col]], test[[col]]])
        fq_encode = temp_df[col].value_counts(dropna=False).to_dict()   
        train[col+'_fq_enc'] = train[col].map(fq_encode)
        test[col+'_fq_enc']  = test[col].map(fq_encode)

    un_corr_cols = []
    for col in train.columns:
        if (train[col].dtype != 'object'):
            if(abs(pearsonr(train[col].fillna(-999), train_y)[1]) > 0.05):
                un_corr_cols.append(col)
            print(col, pearsonr(train[col].fillna(-999), train_y))

    train = fe_dt(train)
    test = fe_dt(test)
    
    train = train.drop(target_encode_book + un_corr_cols, axis=1)
    test = test.drop(target_encode_book + un_corr_cols, axis=1)
    
    return train, test, drop_na_cols + un_corr_cols


train, test, drop_cols = fe(train, test)

65  features are removed...
isFraud (0.9999999999999984, 0.0)
TransactionDT (0.08465765217337731, 6.402371880412322e-61)
TransactionAmt (0.13182413698408704, 9.841877603231417e-146)
card1 (-0.00853935780854106, 0.09731814656297053)
card2 (0.002790249312415651, 0.5879946508493935)
card3 (-0.019806880250285937, 0.0001200621418034677)
card4 (0.023073717576326775, 7.447045819790827e-06)
card5 (-0.002881905863917302, 0.5757931799672842)
card6 (-0.009001241655282394, 0.08051958159601677)
addr1 (0.015005029482161503, 0.003574264646567578)
addr2 (-0.0023632718149925814, 0.6463463930438638)
dist2 (-0.03667747838524305, 1.0518648977810223e-12)
C1 (0.04285128701387919, 8.529391963438241e-17)
C2 (0.04305371439327499, 6.113868316397865e-17)
C4 (0.039359276374390455, 2.0927613211905555e-14)
C6 (0.04030201736541428, 4.954820870303349e-15)
C8 (0.04170370045246905, 5.471489047316919e-16)
C10 (0.043342727010464696, 3.7906089107027954e-17)
C11 (0.04232866055823215, 2.000661939957093e-16)
C13 (0.036293862

V237 (-0.0009610854256230799, 0.8519739409784336)
V238 (-0.003772520929825811, 0.46388804915560883)
V239 (-0.0023228028471713597, 0.6519993474209107)
V240 (-0.0015965272508499848, 0.7565797188666472)
V241 (-0.0015965072457289771, 0.7565826725809139)
V242 (-0.0010029611525931813, 0.8456036677415136)
V243 (-0.0009145447920710253, 0.8590651849845707)
V244 (-0.0010016768074139122, 0.8457988988227491)
V245 (-0.002863096145849858, 0.5782873888636485)
V246 (-0.0009540563647019136, 0.853044183844077)
V247 (-0.001219229463786593, 0.8128735220362496)
V248 (-0.0011222854353733548, 0.8275082984695998)
V249 (-0.001164275981797263, 0.8211616866456497)
V250 (-0.004869887085120173, 0.3443920635868038)
V251 (-0.0044001841770275755, 0.3929245504325881)
V252 (-0.0009646562100028855, 0.8514303585951449)
V253 (-0.000878863760118717, 0.8645095277705458)
V254 (-0.0008843763289232383, 0.8636679728374469)
V255 (-0.0027921995243812754, 0.5877337950463434)
V256 (-0.0015206751134293327, 0.7678043121234981)
V257 (

device_name_fq_enc (-0.023739330331099966, 4.030956992749105e-06)
DeviceType_target_code (0.12688603536581444, 4.418758148286129e-135)
DeviceType_fq_enc (-0.09856603968086913, 5.044877542628302e-82)
id_12_target_code (0.006366086620253614, 0.21644905575954165)
id_12_fq_enc (0.00447154245008694, 0.38529550944193663)
id_13_target_code (0.1178062059166716, 1.3839128289526937e-116)
id_13_fq_enc (-0.07977363430813611, 2.8224257895620924e-54)
id_14_target_code (0.05809586083893349, 1.4840271070152462e-29)
id_14_fq_enc (0.041025085031457034, 1.604619275616194e-15)
id_15_target_code (0.06633132043672117, 4.940702429462678e-38)
id_15_fq_enc (-0.010003199785589705, 0.05210944461367802)
id_16_target_code (0.06857883642262033, 1.5349496098398897e-40)
id_16_fq_enc (-0.020914876359516096, 4.882319596327875e-05)
id_17_target_code (0.07789718615304791, 7.904633449836669e-52)
id_17_fq_enc (-0.021137356912674092, 4.053649872205991e-05)
id_19_target_code (0.2255399761189171, 0.0)
id_19_fq_enc (-0.0655309

In [8]:
train.head()

,isFraud,TransactionDT,TransactionAmt,dist2,D1,D2,D3,D4,D5,D6,D8,D10,D13,D14,D15,V14,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V31,V32,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V50,V55,V56,V59,V60,V61,V62,V63,V64,V65,V66,V67,V71,V72,V77,V78,V80,V81,V82,V83,V84,V85,V86,V87,V88,V92,V93,V111,V112,V113,V117,V118,V119,V120,V121,V122,V139,V140,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V206,V208,V209,V210,V211,V212,V213,V214,V215,V216,V263,V265,V273,V275,V276,V277,V278,V279,V280,V290,V291,V292,V293,V294,V295,V297,V298,V299,V302,V303,V304,V326,V335,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_18,id_21,id_22,id_30,id_31,id_34,DeviceInfo,card1_TransactionAmt_mean,card1_TransactionAmt_std,card2_TransactionAmt_std,card3_TransactionAmt_mean,card5_TransactionAmt_std,uid_TransactionAmt_mean,uid_TransactionAmt_std,uid2_TransactionAmt_mean,uid2_TransactionAmt_std,uid3_TransactionAmt_mean,dist2_isnull,D2_isnull,D3_isnull,D4_isnull,D5_isnull,D6_isnull,D8_isnull,D9_isnull,D10_isnull,D13_isnull,D14_isnull,D15_isnull,V14_isnull,V17_isnull,V18_isnull,V19_isnull,V20_isnull,V21_isnull,V22_isnull,V23_isnull,V24_isnull,V25_isnull,V26_isnull,V31_isnull,V32_isnull,V37_isnull,V38_isnull,V39_isnull,V40_isnull,V41_isnull,V42_isnull,V43_isnull,V44_isnull,V45_isnull,V46_isnull,V47_isnull,V50_isnull,V55_isnull,V56_isnull,V59_isnull,V60_isnull,V61_isnull,V62_isnull,V63_isnull,V64_isnull,V65_isnull,V66_isnull,V67_isnull,V71_isnull,V72_isnull,V77_isnull,V78_isnull,V80_isnull,V81_isnull,V82_isnull,V83_isnull,V84_isnull,V85_isnull,V86_isnull,V87_isnull,V88_isnull,V92_isnull,V93_isnull,id_03_isnull,id_04_isnull,id_07_isnull,id_08_isnull,id_09_isnull,id_10_isnull,id_18_isnull,id_21_isnull,id_22_isnull,id_24_isnull,id_25_isnull,id_26_isnull,addr1_target_code,addr1_fq_enc,C1_target_code,C1_fq_enc,C2_target_code,C2_fq_enc,C4_target_code,C4_fq_enc,C6_target_code,C6_fq_enc,C8_target_code,C8_fq_enc,C10_target_code,C10_fq_enc,C11_target_code,C11_fq_enc,C13_target_code,C13_fq_enc,C14_target_code,C14_fq_enc,card1_target_code,card1_fq_enc,card2_target_code,card2_fq_enc,card3_target_code,card3_fq_enc,card4_target_code,card5_target_code,D9_target_code,D9_fq_enc,device_name_target_code,device_name_fq_enc,DeviceType_target_code,DeviceType_fq_enc,id_13_target_code,id_13_fq_enc,id_14_target_code,id_14_fq_enc,id_15_target_code,id_16_target_code,id_16_fq_enc,id_17_target_code,id_17_fq_enc,id_19_target_code,id_19_fq_enc,id_20_target_code,id_20_fq_enc,id_23_target_code,id_23_fq_enc,id_24_target_code,id_24_fq_enc,id_25_target_code,id_25_fq_enc,id_26_target_code,id_26_fq_enc,id_28_target_code,id_28_fq_enc,id_29_target_code,id_29_fq_enc,id_30_device_target_code,id_30_device_fq_enc,id_30_version_target_code,id_30_version_fq_enc,id_32_target_code,id_32_fq_enc,id_33_target_code,id_33_fq_enc,id_33_0_target_code,id_33_0_fq_enc,id_33_1_target_code,id_33_1_fq_enc,id_33_area_target_code,id_33_area_fq_enc,id_33_ratio_target_code,id_33_ratio_fq_enc,id_35_target_code,id_35_fq_enc,id_37_target_code,id_37_fq_enc,id_38_target_code,id_38_fq_enc,P_emaildomain_target_code,P_emaildomain_fq_enc,R_emaildomain_target_code,R_emaildomain_fq_enc,cents_target_code,cents_fq_enc,DT_month,DT_week_year,DT_day_year,DT_hour,DT_day_week,DT_day,D1_day_mean,D1_week_mean,D2_day_mean,D2_week_mean,D3_day_mean,D3_week_mean,D6_day_mean,D6_week_mean,D7_day_mean,D7_week_mean,D8_day_mean,D8_week_mean,D10_day_mean,D10_week_mean,D13_day_mean,D13_week_mean,D14_day_mean,D14_week_mean,D15_day_mean,D15_week_mean
TransactionID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,

In [9]:
train.to_pickle('../input/mini-model-features/train_features_r.pkl')
test.to_pickle('../input/mini-model-features/test_features_r.pkl')

# POINT_SIZE = 2
# for col in train.columns:
#     fig = plt.figure(figsize=(12,8))

#     plt.scatter(train[train['DT_month'] == 12]['TransactionDT'], train[train['DT_month'] == 12][col], s=POINT_SIZE, c='b')     
#     plt.scatter(train[train['DT_month'] == 13]['TransactionDT'], train[train['DT_month'] == 13][col], s=POINT_SIZE, c='k')
#     plt.scatter(train[train['DT_month'] == 14]['TransactionDT'], train[train['DT_month'] == 14][col], s=POINT_SIZE, c='b')
#     plt.scatter(train[train['DT_month'] == 15]['TransactionDT'], train[train['DT_month'] == 15][col], s=POINT_SIZE, c='k')
#     plt.scatter(train[train['DT_month'] == 16]['TransactionDT'], train[train['DT_month'] == 16][col], s=POINT_SIZE, c='b')
#     plt.scatter(train[train['DT_month'] == 17]['TransactionDT'], train[train['DT_month'] == 17][col], s=POINT_SIZE, c='k')

#     plt.scatter(train[train.isFraud==1]['TransactionDT'], train[train.isFraud==1][col], s=POINT_SIZE * 3, c='r')
    
#     plt.scatter(test['TransactionDT'], test[col], s=POINT_SIZE, c='g')
#     plt.title(col)
#     plt.grid()
# #     plt.show()
    
#     fig.savefig("../eda/mini-models/R/R_" + str(col) + ".png")
#     plt.cla()
#     plt.clf()
#     plt.close(fig)
